In [1]:
import pyspark
import numpy as np

In [20]:

# Specify the path to your Excel file
file_path = "Trade_Area_assignment.xlsx"

import pandas as pd
from pyspark.sql import SparkSession

# Read Excel using pandas
distance = pd.read_excel(file_path, sheet_name="Distance")
census= pd.read_excel(file_path, sheet_name="Census")
stores=pd.read_excel(file_path, sheet_name="stores")

# Initialize SparkSession
spark = SparkSession.builder.appName("ExcelReader").getOrCreate()

# Convert pandas DataFrame to Spark DataFrame
df_distance = spark.createDataFrame(distance)
df_census = spark.createDataFrame(census)
df_stores = spark.createDataFrame(stores)

In [21]:
df_census.show()

+--------------------+----------+-------------------+
|  customer community|Households|Expenditure grocery|
+--------------------+----------+-------------------+
|              Adams |      4000|                800|
|             Arleta |      8000|               1000|
|    Atwater Village |     12000|               1100|
|        Barnes City |      6000|                700|
|            Bel Air |      1000|               1200|
|Benedict-Coldwate...|      7000|                500|
|          Echo Park |      9000|                700|
|          San Pedro |      5000|                900|
|       Sherman Oaks |      4000|               1400|
|      Granada Hills |      5011|               1234|
|   Lakeview Terrace |      4230|               1625|
|      Panorama City |      4708|               1357|
|        Toluca Lake |     10022|               1134|
|      Warner Center |      9645|               1630|
| Wholesale District |     11387|               1367|
+--------------------+------

In [8]:
from pyspark.sql.functions import col
df_census=df_census.withColumn('total',col('Households')*col('Expenditure grocery'))

In [31]:
attractiveness = df_stores.select("customer community").rdd.flatMap(lambda x: x).collect()
possible_store=np.unique(attractiveness)

In [33]:
comm = df_distance.select("customer community").rdd.flatMap(lambda x: x).collect()
community=np.unique(comm)

In [34]:
keys=[(x,y) for x in community for y in possible_store]
print(keys)

[(np.str_('Adams '), np.str_('Echo Park ')), (np.str_('Adams '), np.str_('Granada Hills ')), (np.str_('Adams '), np.str_('Lakeview Terrace ')), (np.str_('Adams '), np.str_('Warner Center ')), (np.str_('Arleta '), np.str_('Echo Park ')), (np.str_('Arleta '), np.str_('Granada Hills ')), (np.str_('Arleta '), np.str_('Lakeview Terrace ')), (np.str_('Arleta '), np.str_('Warner Center ')), (np.str_('Atwater Village '), np.str_('Echo Park ')), (np.str_('Atwater Village '), np.str_('Granada Hills ')), (np.str_('Atwater Village '), np.str_('Lakeview Terrace ')), (np.str_('Atwater Village '), np.str_('Warner Center ')), (np.str_('Barnes City '), np.str_('Echo Park ')), (np.str_('Barnes City '), np.str_('Granada Hills ')), (np.str_('Barnes City '), np.str_('Lakeview Terrace ')), (np.str_('Barnes City '), np.str_('Warner Center ')), (np.str_('Bel Air '), np.str_('Echo Park ')), (np.str_('Bel Air '), np.str_('Granada Hills ')), (np.str_('Bel Air '), np.str_('Lakeview Terrace ')), (np.str_('Bel Air 

In [42]:
from pyspark.ml.feature import MinMaxScaler,VectorAssembler
spark = SparkSession.builder.appName("MinMaxScalerExample").getOrCreate()

scaler = MinMaxScaler(inputCol="features", outputCol="scaled_features")

assembler = VectorAssembler(inputCols=columns_to_scale, outputCol="features")
df_assembled = assembler.transform(df_stores)

# Apply MinMaxScaler to the 'features' column
scaler = MinMaxScaler(inputCol="features", outputCol="scaled_features")
scaler_model = scaler.fit(df_assembled)
df_scaled = scaler_model.transform(df_assembled)

# Show the scaled values
df_scaled.select("customer community", "scaled_features").show()

+------------------+--------------------+
|customer community|     scaled_features|
+------------------+--------------------+
|        Echo Park |[0.28909952606635...|
|    Granada Hills |[0.47867298578199...|
|    Warner Center |[1.0,1.0,0.0,0.0,...|
| Lakeview Terrace |[0.0,0.4545454545...|
+------------------+--------------------+



In [37]:
df_stores.show()

+------------------+----+--------------+--------+-------+-------------+------+-------------------+
|customer community|Size|Parking spaces|Highways|Traffic|Accessibility|Design|Business Comunities|
+------------------+----+--------------+--------+-------+-------------+------+-------------------+
|        Echo Park |1500|           400|       4|      4|           10|     7|                  5|
|    Granada Hills |1900|           150|       8|      8|            5|     6|                  6|
|    Warner Center |3000|           700|       3|      2|            8|     8|                  4|
| Lakeview Terrace | 890|           400|       5|      6|            7|    10|                  3|
+------------------+----+--------------+--------+-------+-------------+------+-------------------+

